# Documentación de Analizadores

Este notebook contiene la documentación y ejemplos de uso de todos los analizadores disponibles en el proyecto.

## Índice

1. [Clasificación de Roles para DataSources](#clasificacion-roles-datasources)
   - Introducción
   - Roles Disponibles
   - Funciones Principales
   - Ejemplos de Uso
   - Extensibilidad

---

*Nota: Este documento se extiende con la documentación de nuevos analizadores a medida que se crean.*


## 1. Clasificación de Roles para DataSources

### Introducción

Sistema flexible y extensible para clasificar las URLs principales de los objetos en `dataSources` y asignarles roles según criterios configurables.

**Módulos principales:**
- `role_classifier.py`: Módulo de clasificación que contiene los criterios para identificar cada tipo de role.
- `datasource_role_classifier.py`: Funciones para clasificar los dataSources y actualizar la base de datos.

### ¿Qué hace este analizador?

Este analizador:
- Clasifica la **URL principal** de cada objeto en `dataSources` (campo `url`)
- Asigna el `role` al objeto completo del dataSource
- **NO modifica** los links dentro de `dataSources[].links`
- Usa `datasource_role()` de `companies_querys.py` para actualizar la DB
- Si no se encuentra un role, **NO agrega** el campo `role` en la DB


### Roles Disponibles (Role_Kind_Companies_Model.txt V2)

El sistema puede clasificar URLs en los siguientes roles:

- **`official_site`**: Sitio oficial de la empresa
- **`official_social_profile`**: Perfil oficial de la empresa en redes sociales
- **`official_social_content`**: Contenido oficial en redes sociales
- **`social_profile`**: Perfiles en redes sociales NO oficiales
- **`social_content`**: Contenido en redes sociales NO oficial
- **`store_listing`**: Tiendas de aplicaciones (Play Store, App Store, etc.)
- **`regulator_profile`**: Ficha/registro de la empresa en un regulador
- **`regulator_reference`**: Páginas de regulación genérica
- **`news_site`**: Portales de noticias y medios
- **`third_party`**: Páginas de terceros que referencian a la empresa
- **`web_utilities`**: Recursos internos (imágenes, rutas internas, videos)
- **`documents`**: URLs para descargar archivos (PDF, Word, etc.)
- **`unclassified`**: URLs que no pueden ser clasificadas en ninguna categoría


### **Funciones Principales**

#### ⚙️ Configuración Notebook

Define el PATH para el correcto funcionamiento del notebook

In [ ]:
# Se asume que el paquete 'src' esta instalado en el entorno (pip install -e .)
import sys
print(f"✅ Python executable: {sys.executable}")


#### Importa modulos y funciones

In [ ]:
# Importar las funciones necesarias
from src.analizers.datasource_role_classifier import (
    classify_company_datasources,
    classify_single_datasource,
    get_datasources_by_role,
    clear_all_company_roles,
    clear_single_datasource_role
)
from src.analizers.role_classifier import get_available_roles

print("✅ Módulos importados correctamente")

#### Define aquí las variables que se usarán en todos los ejemplos:

- `slug`: Slug de la compañía a analizar
- `datasource_url`: URL de un dataSource específico para ejemplos individuales

Puedes modificar estos valores según necesites para tus pruebas.

In [ ]:
# CONFIGURACIÓN - Modifica estos valores según tus necesidades

SLUG = "fraccional"  # Cambiar por el slug de tu compañía
DATASOURCE_URL = "https://fraccional.cl"  # Cambiar por la URL que quieras clasificar

print("✅ Configuración cargada:")
print(f"   📌 Slug: {SLUG}")
print(f"   🔗 DataSource URL: {DATASOURCE_URL}")


#### Roles Disponibles

Primero, veamos qué roles están disponibles en el sistema:

In [ ]:
# Obtener todos los roles disponibles
roles = get_available_roles()

print("Roles disponibles para clasificación:")
print("=" * 50)
for role in sorted(roles):
    print(f"  • {role}")

print(f"\nTotal: {len(roles)} roles disponibles")


#### 1. `classify_company_datasources(slug, target_roles=None)`

Clasifica los roles de **todos los dataSources** de una compañía.

**Parámetros:**
- `slug` (str): Slug de la compañía
- `target_roles` (List[str] | None): Lista de roles a buscar. Si es `None`, busca todos los roles disponibles.

**Retorna:** Dict con estadísticas:
```python
{
    "processed": <int>,      # Total de dataSources procesados
    "classified": <int>,     # DataSources que recibieron un role
    "not_classified": <int>, # DataSources sin role
    "roles_found": {<role>: <count>}, # Contador por role
    "updated": <int>         # Cantidad de dataSources actualizados en la DB
}
```


Ejemplo 1: Clasificar todos los dataSources (todos los roles)

In [ ]:
# Clasificar todos los roles disponibles
stats = classify_company_datasources(
    slug=SLUG,
    target_roles=None  # None = todos los roles
)

print("📊 Estadísticas de clasificación:")
print("=" * 50)
print(f"  DataSources procesados: {stats['processed']}")
print(f"  DataSources clasificados: {stats['classified']}")
print(f"  DataSources sin clasificar: {stats['not_classified']}")
print(f"  DataSources actualizados en DB: {stats['updated']}")
print(f"\n  Roles encontrados:")
for role, count in stats['roles_found'].items():
    print(f"    • {role}: {count}")


Ejemplo 2: Clasificar solo roles específicos

Este ejemplo muestra cómo clasificar solo ciertos roles específicos. Útil cuando solo te interesan ciertos tipos de dataSources.


In [ ]:
# Solo buscar "official_site" y "official_social_profile"
stats = classify_company_datasources(
    slug=SLUG,
    target_roles=["official_site", "official_social_profile"]
)

print("🎯 Clasificación de roles específicos:")
print("=" * 50)
print(f"  Roles encontrados: {stats['roles_found']}")
print(f"  DataSources clasificados: {stats['classified']}")
print(f"  DataSources actualizados: {stats['updated']}")


#### 2. `classify_single_datasource(slug, datasource_url, target_roles=None)`

Clasifica el role de **un dataSource específico**.

**Parámetros:**
- `slug` (str): Slug de la compañía
- `datasource_url` (str): URL del dataSource a clasificar
- `target_roles` (List[str] | None): Lista de roles a buscar. Si es `None`, busca todos.

**Retorna:** Dict con estadísticas:
```python
{
    "processed": <int>,      # Siempre 1 si se encuentra
    "classified": <int>,     # 1 si se clasificó, 0 si no
    "not_classified": <int>, # 0 si se clasificó, 1 si no
    "role": <str|None>,      # Role encontrado o None
    "updated": <bool>        # Si se actualizó la DB
}
```

##### Ejemplo: Clasificar un dataSource específico

In [ ]:
stats = classify_single_datasource(
    slug=SLUG,
    datasource_url=DATASOURCE_URL,
    target_roles=["official_site"]  #None = Todos los roles
)

print("🔍 Clasificación de un dataSource:")
print("=" * 50)
print(f"  URL: {DATASOURCE_URL}")
print(f"  Role encontrado: {stats['role']}")
print(f"  Actualizado en DB: {stats['updated']}")
if stats.get('error'):
    print(f"  ⚠️ Error: {stats['error']}")


#### 3. `get_datasources_by_role(slug, role=None)`

Obtiene dataSources filtrados por role.

**Parámetros:**
- `slug` (str): Slug de la compañía
- `role` (str | None): Role a filtrar. Si es `None`, retorna todos.

**Retorna:** Lista de dicts con formato `[{"url": "...", "role": "..."}, ...]`

Ejemplo: Obtener dataSources filtrados por role

In [ ]:
# Obtener solo dataSources con role "official_site"
official_datasources = get_datasources_by_role(
    slug=SLUG,
    role="official_site"
)

# Obtener todos los dataSources
all_datasources = get_datasources_by_role(slug=SLUG, role=None)
print(f"📊 Total de dataSources: {len(all_datasources)}")
print(f"📋 DataSources con role 'official_site': {len(official_datasources)}")
print("=" * 50)
for i, ds in enumerate(official_datasources, 1):  # Mostrar todos
    print(f"  {i}. {ds['url']} (role: {ds['role']})")


#### 4. `clear_all_company_roles(slug)`

Elimina todos los roles de todos los dataSources de una compañía.
Deja la empresa como si nunca se hubiera procesado para identificar ningún role.

**Parámetros:**
- `slug` (str): Slug de la compañía

**Retorna:** Dict con estadísticas:
```python
{
    "processed": <int>,  # Total de dataSources procesados
    "cleared": <int>,     # DataSources que tenían role y fueron limpiados
    "not_found": <int>,   # DataSources que no tenían role
    "updated": <int>      # Cantidad de dataSources actualizados en la DB
}
```


##### Ejemplo: Limpiar todos los roles de una empresa

In [ ]:
# Limpiar todos los roles de la compañía
stats = clear_all_company_roles(slug=SLUG)

print("🧹 Estadísticas de limpieza:")
print("=" * 50)
print(f"  DataSources procesados: {stats['processed']}")
print(f"  DataSources limpiados: {stats['cleared']}")
print(f"  DataSources sin role: {stats['not_found']}")
print(f"  DataSources actualizados en DB: {stats['updated']}")
if stats.get('error'):
    print(f"  ⚠️ Error: {stats['error']}")


#### 5. `clear_single_datasource_role(slug, datasource_url)`


Elimina el role de un dataSource específico de una compañía.
Limpia solo una URL, no todo el dataSource.

**Parámetros:**
- `slug` (str): Slug de la compañía
- `datasource_url` (str): URL del dataSource a limpiar

**Retorna:** Dict con estadísticas:
```python
{
    "processed": <int>,  # Siempre 1 si se encuentra
    "cleared": <int>,     # 1 si tenía role y fue limpiado, 0 si no
    "not_found": <int>,   # 0 si tenía role, 1 si no tenía
    "updated": <bool>     # Si se actualizó la DB
}
```


##### Ejemplo: Limpiar el role de un dataSource específico


In [ ]:
# Limpiar el role de un dataSource específico
stats = clear_single_datasource_role(
    slug=SLUG,
    datasource_url=DATASOURCE_URL
)

print("🧹 Limpieza de un dataSource:")
print("=" * 50)
print(f"  URL: {DATASOURCE_URL}")
print(f"  Limpiado: {stats['cleared'] == 1}")
print(f"  Actualizado en DB: {stats['updated']}")
if stats.get('error'):
    print(f"  ⚠️ Error: {stats['error']}")


### Estructura de Datos

Antes de la clasificación

Los dataSources pueden tener o no el campo `role`:

```json
{
  "dataSources": [
    {
      "url": "https://realblocks.com",
      "links": {...},
      "texts": {...}
    },
    {
      "url": "https://linkedin.com/company/realblocks",
      "links": {...},
      "texts": {...}
    }
  ]
}
```

Después de la clasificación.

Los dataSources clasificados tienen el campo `role`:

```json
{
  "dataSources": [
    {
      "url": "https://realblocks.com",
      "role": "official_site",
      "links": {...},
      "texts": {...}
    },
    {
      "url": "https://linkedin.com/company/realblocks",
      "role": "official_social_profile",
      "links": {...},
      "texts": {...}
    }
  ]
}
```


### Comportamiento del Sistema

1. **Si no se encuentra un role**: El campo `role` **NO se agrega** al dataSource en la DB.
2. **Si se encuentra un role**: Se actualiza el campo `role` del dataSource usando `datasource_role()` de `companies_querys.py`.
3. **Si se especifican `target_roles`**: Solo se buscan esos roles específicos. Los dataSources que no cumplan con esos roles no se modifican.
4. **Si `target_roles` es `None`**: Se evalúan todos los roles disponibles excepto "unclassified". "unclassified" solo se aplica si está explícitamente en `target_roles`.


### Extensibilidad

El sistema es completamente extensible. Puedes agregar nuevos roles o modificar los existentes.

#### Agregar un nuevo role

In [ ]:
from src.analizers.role_classifier import register_custom_classifier

def mi_nuevo_classificador(url: str, primary_domain: str) -> bool:
    """Define la lógica para identificar el nuevo role."""
    # Tu lógica aquí
    return "mi-dominio.com" in url.lower()

# Registrar el clasificador
register_custom_classifier("mi_nuevo_role", mi_nuevo_classificador)

#### Modificar un clasificador existente

In [ ]:
from src.analizers.role_classifier import _default_classifier, remove_classifier

# Eliminar un clasificador específico
def old_classifier(url, domain):
    return False

remove_classifier("official_site", old_classifier)

# Agregar uno nuevo
def new_classifier(url, domain):
    # Nueva lógica
    return True

_default_classifier.register_classifier("official_site", new_classifier)

In [ ]:
# Ejemplo de extensibilidad: Agregar un clasificador personalizado
from src.analizers.role_classifier import register_custom_classifier

def ejemplo_classificador_personalizado(url: str, primary_domain: str) -> bool:
    """
    Ejemplo de clasificador personalizado.
    Este ejemplo identifica URLs que contienen 'ejemplo' en el dominio.
    """
    url_lower = url.lower()
    return "ejemplo" in url_lower

# Descomentar para registrar el clasificador
# register_custom_classifier("ejemplo_role", ejemplo_classificador_personalizado)
# print("✅ Clasificador personalizado registrado")

print("💡 Para usar este ejemplo, descomenta las líneas anteriores")


---
## 2. Próximos Analizadores
*Esta sección se extenderá con la documentación de nuevos analizadores a medida que se creen.*


### Cómo agregar un nuevo analizador a esta documentación

1. Agregar una nueva sección numerada (ej: "## 2. Nombre del Analizador")
2. Incluir:
   - Introducción y propósito
   - Funciones principales con documentación
   - Ejemplos de uso ejecutables
   - Estructura de datos si aplica
   - Notas de comportamiento

3. Actualizar el índice al inicio del notebook

---

*Última actualización: 2025-01-XX*